## Analyzing CIA Factbook Data Using SQL

In this project, I'll be answering the following questions

1. What is the minimum and maximum population? What is the average poplation and average area?
2. Which country has the highest population and which country has the lowest population?
3. Which country has the most people? Which country has the highest growth rate?
4. Which countries have the highest ratios of water to land? Which countries have more water than land?
5. Which countries will add the most people to their populations next year?
6. Which countries have a higher death rate than birth rate?
7. Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?


In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT * FROM facts 
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [3]:
%%sql
SELECT MIN(population), 
    MAX(population),
    MIN(population_growth),
    MAX(population_growth)
FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


In [4]:
%%sql
--retrieving country with the lowest population
SELECT *
    FROM facts
WHERE population = (SELECT MIN(population) 
                    FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [5]:
%%sql
--retrieving country with the highest population
SELECT *
    FROM facts
WHERE population = (SELECT MAX(population) 
                    FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


In [6]:
%%sql
--Recomputing the summary statistics to exclude the row for the whole world population
SELECT MIN(population) AS Min_popultion,
    MAX(population) AS Max_population,
    MIN(population_growth) AS Min_population_growth,
    MAX(population_growth) AS MAX_population_growth
FROM facts
WHERE population <> (SELECT MAX(population)
                     FROM(facts));

Done.


Min_popultion,Max_population,Min_population_growth,MAX_population_growth
0,1367485388,0.0,4.02


In [7]:
%%sql
--Calculating the average population and average area
SELECT ROUND(AVG(population), 3) AS Avg_Population,
    ROUND(AVG(area), 3) AS Avg_Area
FROM facts
WHERE name <> 'world';

Done.


Avg_Population,Avg_Area
62094928.322,555093.546


In [8]:
%%sql
--Identifying countries that are above the average population and below area
SELECT *
FROM facts
WHERE population > (SELECT AVG(population)
                    FROM facts
                    WHERE name <> 'World'
                   )
AND area < (SELECT AVG(area)
            FROM facts
                WHERE name <> 'World');


Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


In [9]:
%%sql
--Country which is highly populated
SELECT name AS country, MAX(population) AS population
FROM facts
WHERE name <> 'World';

Done.


country,population
China,1367485388


In [10]:
%%sql
--Country with the highest growth rate
with world_info AS (
    SELECT name AS country, 
        population, 
        population_growth, 
        birth_rate, 
        death_rate
    FROM facts
    WHERE name NOT LIKE '__r%d'
)

SELECT country, 
    population_growth AS population_growth
FROM world_info
WHERE population_growth <> 'None'
ORDER BY population_growth
LIMIT 5;

Done.


country,population_growth
Holy See (Vatican City),0.0
Cocos (Keeling) Islands,0.0
Greenland,0.0
Pitcairn Islands,0.0
Greece,0.01


In [11]:
%%sql
--country with the highest population gowth
with world_info AS (
    SELECT name AS country, 
        population, 
        population_growth, 
        birth_rate, 
        death_rate
    FROM facts
    WHERE name NOT LIKE '__r%d'
)

SELECT country, 
    population, 
    population_growth AS population_growth,
    birth_rate
FROM world_info
ORDER BY population_growth DESC
LIMIT 5;

Done.


country,population,population_growth,birth_rate
South Sudan,12042910,4.02,36.91
Malawi,17964697,3.32,41.56
Burundi,10742276,3.28,42.01
Niger,18045729,3.25,45.45
Uganda,37101745,3.24,43.79


In [12]:
%%sql
--countries with the highest ratios of water to land
SELECT name AS country,
    (area_land/area_water) AS water_to_land_ratio
FROM facts
ORDER BY water_to_land_ratio DESC
LIMIT 5;

Done.


country,water_to_land_ratio
Bosnia and Herzegovina,5118
Niger,4222
Morocco,1785
Guinea,1755
Costa Rica,1276


In [13]:
%%sql
--Countries with more water than land
SELECT name AS country, 
    area_water,
    area_land
FROM facts
WHERE area_water > area_land 
LIMIT 5;

Done.


country,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


In [14]:
%%sql
--Countries that will add the most people to their populations next year
SELECT name AS country,
    population,
    birth_rate
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts
                   )
    AND birth_rate > (SELECT AVG(birth_rate)
                     FROM facts
                     )
    AND name NOT LIKE '__r%d'
ORDER BY birth_rate DESC
LIMIT 5;

Done.


country,population,birth_rate
Nigeria,181562056,37.64
Ethiopia,99465819,37.27
"Congo, Democratic Republic of the",79375136,34.88
Philippines,100998376,24.27
Egypt,88487396,22.9


In [15]:
%%sql
--countries with higher death rate than birth rate
SELECT name AS country,
    death_rate,
    birth_rate
FROM facts
WHERE death_rate > birth_rate
ORDER BY death_rate DESC
LIMIT 5;

Done.


country,death_rate,birth_rate
Ukraine,14.46,10.72
Bulgaria,14.44,8.92
Latvia,14.31,10.0
Lithuania,14.27,10.1
Russia,13.69,11.6


In [16]:
%%sql
--Countries that have the highest population to area ratio
SELECT name AS country,
    (population/area) AS population_area_ratio
FROM facts
ORDER BY population_area_ratio DESC
LIMIT 5;

Done.


country,population_area_ratio
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
